# Number Partitioning with `Yao.jl`

In [ ]:
using Yao
import YaoPlots
using PyCall, PyPlot
np = pyimport("numpy");
PyPlot.plt.style.use("../paper.mplstyle")

using LinearAlgebra, StatsBase, Distributions
using LsqFit, Measurements
using Parameters, Printf, HDF5, JLD

In [ ]:
Base.:+(s1::String, s2::String) = s1 * s2
PATH = "../data/number-partitioning/"
BATCH = "/batch_06/"

INPUT_FOLDER = PATH + "instances" + BATCH
OUTPUT_FOLDER = PATH + "results" + BATCH

In [ ]:
p_range = [1, 3, 5, 7]
X = 4:2:20

ENSEMBLE_SIZE = 100;

In [ ]:
all_costs = Dict()

for n in X
    num_qubits = n-1
    all_costs[n] = []
    
    for i in 0:ENSEMBLE_SIZE-1
        
        filename = (@sprintf "number_partitioning_num_%.02d_replica_%.03d" n i+1)
        if  !(isfile(OUTPUT_FOLDER + "yao/yao_" + filename + ".h5"))
            continue
        end
        printstyled("loading " + filename + "\n", color=:magenta)
        
        a = np.load(INPUT_FOLDER + filename + ".npy")
        
        J = -2 * np.outer(a |> transpose, a)
        np.fill_diagonal(J, 0.)    
        
        # freeze final spin to +1
        h_z = J[end, 1:end-1]
        J = J[1:end-1, 1:end-1]
        
        all_costs_tmp = []
        c = h5open(OUTPUT_FOLDER + "yao/yao_" + filename + ".h5", "r") do file
            for p in p_range
                try
                    cost = read(file, "p = " + string(p) + "/" + "cost")
                    push!(all_costs_tmp, cost - sum(a.^2))
                catch err
                    println(typeof(err), ", p = ", string(p), " not yet done!")
                end
            end # end p-loop
        end # end do file
        if size(all_costs_tmp)[1] < 3
            continue
        end
        push!(all_costs[n], all_costs_tmp)
        flush(stdout)
    end
end

In [ ]:
cost_for_all_p = [([-all_costs[n][i] .|> sqrt for i in 1:length(all_costs[n])] |> sum) / length(all_costs[n]) for n in X];
cost_for_all_p_square  = [([-all_costs[n][i] for i in 1:length(all_costs[n])] |> sum) / length(all_costs[n]) for n in X];
var_for_all_p  = cost_for_all_p_square .- map(x -> x.^2, cost_for_all_p);

In [ ]:
cost_for_all_p = [cost_for_all_p[n][p] for n in 1:length(X), p in 1:length(p_range)];
std_for_all_p = [var_for_all_p[n][p] .|> sqrt for n in 1:length(X), p in 1:length(p_range)];

## Plots

In [ ]:
labels = ["\$p="+string(p_val)+"\$" for p_val in p_range]
# styles = ["p-C0", "s-C1", "o-C2", "x-C3"];
# colors = ["C3", "C0", "C1", "C2", ];

In [ ]:
fit_params = []
m(x, p) = p[1] .+ p[2] .* x
p0 = [0., 1.]
start = 1
final = 9
for l in 1:length(p_range)
    data_points = cost_for_all_p[start:final, l] 
    fit = curve_fit(m, log.(X[start:final]), log.(data_points), p0)    
    push!(fit_params, fit.param)
    println(fit.resid .|> abs |> sum)    
    data_points = measurement.(cost_for_all_p[start:final, l], std_for_all_p[start:final, l])
    M = hcat(ones(length(X[start:final])), log.(X[start:final]))
    println(M \ log.(data_points))
end

In [ ]:
fit_params

In [ ]:
@pyimport matplotlib.ticker as ticker

styles = ["v", "^", "s", "o"];
colors = ["C3", "#000C8F", "grey", "#0A92C4"];
fig = figure(figsize=(4.2, 2.2))

ax = subplot(111)
for l in 2:length(p_range)
    ax.plot(X[start:final], cost_for_all_p[start:final, l], styles[l], c=colors[l], label=labels[l])
    ax.plot(X[start:final], m(X[start:final] .|> log, fit_params[l]) .|> exp, c=colors[l])#, label=@sprintf("\$N^{%0.2f}\$", round(fit_params[l][2], digits=2)))
end
ax.legend(loc="lower right", ncol=3, columnspacing=1, labelspacing=0)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim(4, 20)
ax.set_ylim(1e-1, 1e0)
ax.set_xticks([k for k in 4:2:20])
ax.set_yticks([1e-1, 1e0])
ax.set_xlabel("\$N\$")
ax.set_ylabel("\$\\langle \\sqrt{E} \\rangle\$", labelpad=-10)

ax.xaxis.set_major_locator(ticker.FixedLocator([k for k in 4:2:20]))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
ax.xaxis.set_minor_locator(ticker.NullLocator())
# ax.yaxis.set_minor_locator(ticker.NullLocator())
ax.set_yticklabels([], minor=true)

tight_layout(pad=0.15, w_pad=0.0, h_pad=0.0)
savefig("../plots/" * "Fig6.pdf", dpi=300)

## Testing